In [1]:
import os
import numpy as np
import pickle

In [31]:
fashioniq_vector_file_names =  ["dress_one-hot.pickle", "shirt_one-hot.pickle", "toptee_one-hot.pickle"]

In [4]:
fashioniq_imgname_to_vector = {}
for f_name in fashioniq_vector_file_names:
    f_path = os.path.join("/home/deokhk/coursework/CIGAR/data/image_retrieval", f_name)
    with open(f_path, "rb") as f:
        vectors = pickle.load(f)
    fashioniq_imgname_to_vector = {**fashioniq_imgname_to_vector, **vectors}


In [32]:
from scipy.spatial import distance

def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    return nodes[closest_index]


In [33]:
lower_vector_file_name = "lower_one-hot.pickle"
vector_path = os.path.join("/home/deokhk/coursework/CIGAR/data/image_retrieval", lower_vector_file_name)
with open(vector_path, "rb") as f:
    lower_vectors = pickle.load(f)


In [9]:
import json

fashioniq_dir = "/home/deokhk/coursework/fashion-iq/data/captions/"
file_list = ["cap.dress.train.json", "cap.dress.val.json", "cap.shirt.train.json", "cap.shirt.val.json", "cap.toptee.train.json", "cap.toptee.val.json"]
captions = []
for file_name in file_list:
    path = os.path.join(fashioniq_dir, file_name)
    with open(path ,'r') as f:
        cap = json.load(f)
    captions += cap


In [35]:
from tqdm import tqdm

lower_vector_to_name = {v.tostring(): k for k, v in lower_vectors.items()}

lower_vector_list = []
for k ,v in lower_vectors.items():
    lower_vector_list.append(v)

# Returned generated data has a format like..
# [{"target":, "candidate":, "captions:" [..], "similarity": []}]
lower_augmented_caption_datas = []
for caption in tqdm(captions):
    discard = False
    target = caption['target']
    candidate = caption['candidate']
    fashioniq_captions = caption['captions']

    try:
        target_vector = fashioniq_imgname_to_vector[target]
    except KeyError:
        discard = True
        
    try:
        candidate_vector = fashioniq_imgname_to_vector[candidate]
    except KeyError:
        discard = True

    if not discard:
        target_closest_vector = closest_node(target_vector, lower_vector_list)
        candidate_closest_vector = closest_node(candidate_vector, lower_vector_list)
        lower_target_imageid = lower_vector_to_name[target_closest_vector.tostring()]
        candidate_target_imageid = lower_vector_to_name[candidate_closest_vector.tostring()]

        vector_diff_fashioniq = target_vector - candidate_vector
        vector_diff_lower = target_closest_vector - candidate_closest_vector

        similarity = distance.cosine(vector_diff_fashioniq, vector_diff_lower)

        generated_data_pair = dict()
        generated_data_pair['target'] = lower_target_imageid
        generated_data_pair['candidate'] = candidate_target_imageid
        generated_data_pair['captions'] = fashioniq_captions
        generated_data_pair['similarity'] = similarity
        lower_augmented_caption_datas.append(generated_data_pair)


/home/deokhk/anaconda3/envs/fashioniq/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  This is separate from the ipykernel package so we can avoid doing imports until
  0%|          | 0/24016 [00:00<?, ?it/s]/home/deokhk/anaconda3/envs/fashioniq/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/deokhk/anaconda3/envs/fashioniq/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  0%|          | 13/24016 [00:08<4:11:59,  1.59it/s]


In [37]:
lower_augmented_caption_datas[5:]

[{'target': 'img/Cuffed_Origami-Front_Joggers/img_00000004.jpg',
  'candidate': 'img/Lady_Lace_Mini_Skirt/img_00000027.jpg',
  'captions': ['is longer and black wit a fitted waist',
   'is black with straps'],
  'similarity': 0.5256583509747431},
 {'target': 'img/Rose_Print_Leggings/img_00000010.jpg',
  'candidate': 'img/Space_Dye_Capri_Leggings/img_00000065.jpg',
  'captions': ['has long sleeves and stripes', 'is striped with sleeves'],
  'similarity': 0.6150998205402495},
 {'target': 'img/Striped_Linen_Belted_Shorts/img_00000014.jpg',
  'candidate': 'img/Dark_Wash_Jeggings/img_00000026.jpg',
  'captions': ['is longer and yellow',
   'belted maxi dress in block colors of yellow and green'],
  'similarity': 0.18350341907227397},
 {'target': 'img/Coated_Clean_Wash_-_Skinny_Jeans/img_00000011.jpg',
  'candidate': 'img/Striped_Tribal_Print_Sweatpants/img_00000001.jpg',
  'captions': ['is a back patterned dress', 'is not as bright in color'],
  'similarity': 0.3453463292920229},
 {'target'

In [ ]:
import pandas as pd
def describe_threshold_distribution(augmented_lower_dataset):
    thersholds = []
    for elem in augmented_lower_dataset:
        thersholds.append(elem['similarity'])
    df = pd.DataFrame(thersholds)
    df.describe()

describe_threshold_distribution